In [1]:
# WITHOUT PADDING
import torch
sample = 0.5

# WE ARE ON DATALOADER STAGE
arr1 = torch.tensor([2,4,6,8,10])#.unsqueeze(-1)
num_sample = int(len(arr1) * sample)

# Index for shuffle and revert
noise = torch.rand(arr1.shape)
shuffle = torch.argsort(noise, dim=0)
revert = torch.argsort(shuffle, dim=0)

# Get keep value
idx_keep = shuffle[:num_sample]
masked_input = torch.gather(arr1, index=idx_keep, dim=0)
print("ids_keep:", idx_keep)
print("masked_input:", masked_input) # Order of masked input does not matter since we already pos_encoded the sequence
print("_"*100)

# Revert
final_input = torch.cat([masked_input, torch.zeros(len(arr1) - num_sample)])
print("revert:", revert)
print("final_input:", final_input)
torch.gather(final_input, index=revert, dim=0)

ids_keep: tensor([1, 0])
masked_input: tensor([4, 2])
____________________________________________________________________________________________________
revert: tensor([1, 0, 4, 2, 3])
final_input: tensor([4., 2., 0., 0., 0.])


tensor([2., 4., 0., 0., 0.])

In [2]:
# WITH PADDING
max_seq_len = 10
mask_value = 0
padding_idx = max_seq_len

sample = 0.5
arr1 = torch.tensor([2,4,6,8,10])#.unsqueeze(-1)
num_sample = int(len(arr1) * sample)

# Index for shuffle and revert
noise = torch.rand(arr1.shape)
shuffle = torch.argsort(noise, dim=0)
revert = torch.argsort(shuffle, dim=0)

# Get keep value
idx_keep = shuffle[:num_sample]
masked_input = torch.gather(arr1, index=idx_keep, dim=0)
print("ids_keep:", idx_keep)
print("masked_input:", masked_input) # Order of masked input does not matter since we already pos_encoded the sequence

# Padding
padded_input = torch.concat([masked_input, torch.zeros(max_seq_len - len(masked_input))+mask_value])
print("padded_input:", padded_input)
print("_"*100)

# Revert
revert = torch.cat([revert, torch.zeros(max_seq_len-len(revert)) + (max_seq_len-1)]).to(torch.int64)
print("revert:", revert)
res = torch.gather(padded_input, index=revert, dim=0)
res

ids_keep: tensor([3, 2])
masked_input: tensor([8, 6])
padded_input: tensor([8., 6., 0., 0., 0., 0., 0., 0., 0., 0.])
____________________________________________________________________________________________________
revert: tensor([3, 4, 1, 0, 2, 9, 9, 9, 9, 9])


tensor([0., 0., 6., 8., 0., 0., 0., 0., 0., 0.])

In [3]:
# MULTIPLE DIMENSTIONS
# WITH PADDING
mask_value = 0

sample = 0.9
arr1 = torch.tensor([2,4])
arr2 = torch.tensor([2,4,6])
arr3 = torch.tensor([2,4,6,8])
arr4 = torch.tensor([2,4,6,8,10])

arr_li = [arr1, arr2, arr3, arr4]
revert_li = []
masked_input_li = []
max_seq_len = 0

for i in range(len(arr_li)):
    num_sample = int(len(arr_li[i]) * sample)

    # Index for shuffle and revert
    noise = torch.rand(arr_li[i].shape)
    shuffle = torch.argsort(noise, dim=0)
    revert = torch.argsort(shuffle, dim=0)
    max_seq_len = len(revert) if len(revert)>max_seq_len else max_seq_len
    revert_li.append(revert)
    

    # Get keep value
    idx_keep = shuffle[:num_sample]
    masked_input = torch.gather(arr_li[i], index=idx_keep, dim=0)
    masked_input_li.append(masked_input)
    print("ids_keep:", idx_keep)
    print("masked_input:", masked_input) # Order of masked input does not matter since we already pos_encoded the sequence
    print("-"*50)

# Padding
padded_input = torch.nn.utils.rnn.pad_sequence(masked_input_li, batch_first=True)
revert = torch.nn.utils.rnn.pad_sequence(revert_li, batch_first=True, padding_value=max_seq_len-1)
print("padded_input:\n", padded_input)
print("revert:\n", revert)
print("_"*100)

# Revert
print("max_seq_len:", max_seq_len)
filled_input = torch.concat([padded_input, torch.zeros(padded_input.shape[0], (max_seq_len - padded_input.shape[1]+1))], dim=1)
print("filled_input:\n", filled_input)
torch.gather(filled_input, index=revert, dim=-1)

ids_keep: tensor([0])
masked_input: tensor([2])
--------------------------------------------------
ids_keep: tensor([1, 0])
masked_input: tensor([4, 2])
--------------------------------------------------
ids_keep: tensor([1, 0, 2])
masked_input: tensor([4, 2, 6])
--------------------------------------------------
ids_keep: tensor([0, 2, 3, 4])
masked_input: tensor([ 2,  6,  8, 10])
--------------------------------------------------
padded_input:
 tensor([[ 2,  0,  0,  0],
        [ 4,  2,  0,  0],
        [ 4,  2,  6,  0],
        [ 2,  6,  8, 10]])
revert:
 tensor([[0, 1, 4, 4, 4],
        [1, 0, 2, 4, 4],
        [1, 0, 2, 3, 4],
        [0, 4, 1, 2, 3]])
____________________________________________________________________________________________________
max_seq_len: 5
filled_input:
 tensor([[ 2.,  0.,  0.,  0.,  0.,  0.],
        [ 4.,  2.,  0.,  0.,  0.,  0.],
        [ 4.,  2.,  6.,  0.,  0.,  0.],
        [ 2.,  6.,  8., 10.,  0.,  0.]])


tensor([[ 2.,  0.,  0.,  0.,  0.],
        [ 2.,  4.,  0.,  0.,  0.],
        [ 2.,  4.,  6.,  0.,  0.],
        [ 2.,  0.,  6.,  8., 10.]])

In [1]:
import torch
# MULTIPLE DIMENSTIONS
# WITH PADDING
mask_value = 0

sample = 0.5
arr1 = torch.tensor([2,4])
arr2 = torch.tensor([2,4,6])
arr3 = torch.tensor([2,4,6,8])
arr4 = torch.tensor([2,4,6,8,10])

arr_li = [arr1, arr2, arr3, arr4]
revert_li = []
keep_li = []
max_seq_len = 0

for i in range(len(arr_li)):
    num_sample = int(len(arr_li[i]) * sample)

    # Index for shuffle and revert
    noise = torch.rand(arr_li[i].shape)
    shuffle = torch.argsort(noise, dim=0)
    revert = torch.argsort(shuffle, dim=0)
    max_seq_len = len(revert) if len(revert)>max_seq_len else max_seq_len
    revert_li.append(revert)

    # Get keep value
    idx_keep = shuffle[:num_sample]
    keep_li.append(idx_keep)

# Padding
arr = torch.nn.utils.rnn.pad_sequence(arr_li, batch_first=True)
keep = torch.nn.utils.rnn.pad_sequence(keep_li, batch_first=True, padding_value=max_seq_len-1)
revert = torch.nn.utils.rnn.pad_sequence(revert_li, batch_first=True, padding_value=max_seq_len-1)
print("arr:\n", arr)
print("keep_idx:\n", keep)
print("revert_idx:\n", revert)
print("_"*100)

# Select
# Todo: Linear trnasform & pos_encoding
masked_input = torch.gather(arr, index=keep, dim=-1)
print("masked_input:\n", masked_input)

# Revert
filled_input = torch.concat([masked_input, torch.zeros(masked_input.shape[0], (max_seq_len - masked_input.shape[1]+1))], dim=1)
print("filled_input:\n", filled_input)
print()
torch.gather(filled_input, index=revert, dim=-1)

arr:
 tensor([[ 2,  4,  0,  0,  0],
        [ 2,  4,  6,  0,  0],
        [ 2,  4,  6,  8,  0],
        [ 2,  4,  6,  8, 10]])
keep_idx:
 tensor([[0, 4],
        [2, 4],
        [1, 3],
        [1, 4]])
revert_idx:
 tensor([[0, 1, 4, 4, 4],
        [2, 1, 0, 4, 4],
        [3, 0, 2, 1, 4],
        [2, 0, 4, 3, 1]])
____________________________________________________________________________________________________
masked_input:
 tensor([[ 2,  0],
        [ 6,  0],
        [ 4,  8],
        [ 4, 10]])
filled_input:
 tensor([[ 2.,  0.,  0.,  0.,  0.,  0.],
        [ 6.,  0.,  0.,  0.,  0.,  0.],
        [ 4.,  8.,  0.,  0.,  0.,  0.],
        [ 4., 10.,  0.,  0.,  0.,  0.]])



tensor([[ 2.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  6.,  0.,  0.],
        [ 0.,  4.,  0.,  8.,  0.],
        [ 0.,  4.,  0.,  0., 10.]])

In [1]:
# Do we need special mask token?

import torch
# MULTIPLE DIMENSTIONS
# WITH PADDING
mask_value = 99
padding_value = -99

sample = 0.5
arr1 = torch.tensor([2,4])
arr2 = torch.tensor([2,4,6])
arr3 = torch.tensor([2,4,6,8])
arr4 = torch.tensor([2,4,6,8,10])

arr_li = [arr1, arr2, arr3, arr4]

keep_li = []
keep_mask_li = []

revert_li = []
revert_mask_li = []

for i in range(len(arr_li)):
    num_sample = int(len(arr_li[i]) * sample)

    # Index for shuffle and revert
    noise = torch.rand(arr_li[i].shape)
    shuffle = torch.argsort(noise, dim=0)
    revert = torch.argsort(shuffle, dim=0)
    revert_li.append(revert)
    revert_mask_li.append(torch.ones(revert.shape))

    # Get keep value
    idx_keep = shuffle[:num_sample]
    keep_li.append(idx_keep)
    keep_mask_li.append(torch.ones(idx_keep.shape))

# Padding
arr = torch.nn.utils.rnn.pad_sequence(arr_li, batch_first=True)
keep = torch.nn.utils.rnn.pad_sequence(keep_li, batch_first=True)
keep_mask = torch.nn.utils.rnn.pad_sequence(keep_mask_li, batch_first=True)
revert = torch.nn.utils.rnn.pad_sequence(revert_li, batch_first=True)
revert_mask = torch.nn.utils.rnn.pad_sequence(revert_mask_li, batch_first=True)

print("arr:\n", arr)
print("keep:\n", keep)
print("revert_idx:\n", revert)
print("_"*100)

# Select
# Todo: Linear trnasform & pos_encoding
masked_input = torch.gather(arr, index=keep, dim=-1)
masked_input = torch.where(keep_mask==1, masked_input, mask_value)
print("masked_input:\n", masked_input)
print("keep_mask\n", keep_mask)
print("_"*100)

# Revert
filled_input = torch.concat([masked_input, torch.zeros(masked_input.shape[0], (revert_mask.shape[1] - masked_input.shape[1]+1))+mask_value], dim=1)
print("filled_input:\n", filled_input)
print("_"*100)
res = torch.gather(filled_input, index=revert, dim=-1)
print(res)
torch.where(revert_mask == 1, res, padding_value)

arr:
 tensor([[ 2,  4,  0,  0,  0],
        [ 2,  4,  6,  0,  0],
        [ 2,  4,  6,  8,  0],
        [ 2,  4,  6,  8, 10]])
keep:
 tensor([[1, 0],
        [2, 0],
        [1, 2],
        [0, 4]])
revert_idx:
 tensor([[1, 0, 0, 0, 0],
        [2, 1, 0, 0, 0],
        [2, 0, 1, 3, 0],
        [0, 3, 4, 2, 1]])
____________________________________________________________________________________________________
masked_input:
 tensor([[ 4, 99],
        [ 6, 99],
        [ 4,  6],
        [ 2, 10]])
keep_mask
 tensor([[1., 0.],
        [1., 0.],
        [1., 1.],
        [1., 1.]])
____________________________________________________________________________________________________
filled_input:
 tensor([[ 4., 99., 99., 99., 99., 99.],
        [ 6., 99., 99., 99., 99., 99.],
        [ 4.,  6., 99., 99., 99., 99.],
        [ 2., 10., 99., 99., 99., 99.]])
____________________________________________________________________________________________________
tensor([[99.,  4.,  4.,  4.,  4.],
 

tensor([[ 99.,   4., -99., -99., -99.],
        [ 99.,  99.,   6., -99., -99.],
        [ 99.,   4.,   6.,  99., -99.],
        [  2.,  99.,  99.,  99.,  10.]])

In [5]:
# Do we need special mask token?

import torch
# MULTIPLE DIMENSTIONS
# WITH PADDING
mask_value = 99
padding_value = -99

rto = 0.5
arr1 = torch.tensor([2,4])
arr2 = torch.tensor([2,4,6])
arr3 = torch.tensor([2,4,6,8])
arr4 = torch.tensor([2,4,6,8,10])

arr_li = [arr1, arr2, arr3, arr4]

remain_idx_li = []
remain_padding_mask_li = []

revert_idx_li = []
revert_padding_mask_li = []

masked_idx_li = []
masked_padding_mask_li = []

for i in range(len(arr_li)):
    num_remain = int(len(arr_li[i]) * rto)

    # Index for shuffle and revert
    noise = torch.rand(arr_li[i].shape)
    shuffle = torch.argsort(noise, dim=0)
    
    # Get revert
    revert_idx = torch.argsort(shuffle, dim=0)
    revert_idx_li.append(revert_idx)
    revert_padding_mask_li.append(torch.ones(revert_idx.shape))

    # Get remain
    remain_idx = shuffle[:num_remain]
    remain_idx_li.append(remain_idx)
    remain_padding_mask_li.append(torch.ones(remain_idx.shape))
    
    # Get mask 
    masked_idx = shuffle[num_remain:]
    masked_idx_li.append(masked_idx)
    masked_padding_mask_li.append(torch.ones(masked_idx.shape))

# Padding
arr = torch.nn.utils.rnn.pad_sequence(arr_li, batch_first=True)
remain_idx = torch.nn.utils.rnn.pad_sequence(remain_idx_li, batch_first=True)
remain_padding_mask = torch.nn.utils.rnn.pad_sequence(remain_padding_mask_li, batch_first=True)
revert_idx = torch.nn.utils.rnn.pad_sequence(revert_idx_li, batch_first=True)
revert_padding_mask = torch.nn.utils.rnn.pad_sequence(revert_padding_mask_li, batch_first=True)
masked_idx = torch.nn.utils.rnn.pad_sequence(masked_idx_li, batch_first=True)
masked_padding_mask = torch.nn.utils.rnn.pad_sequence(masked_padding_mask_li, batch_first=True)


def apply_remain(data, remain_idx, remain_padding_mask):
    remain_data = torch.gather(data, index=remain_idx, dim=-1)
    remain_data = torch.where(remain_padding_mask==1, remain_data, 99)
    return remain_data
        
print("arr:\n", arr)
remain_arr = apply_remain(arr, remain_idx, remain_padding_mask)
print("_"*100)
print("remain_arr:\n", remain_arr)

def apply_revert(remain_data, remain_padding_mask, masked_idx, revert_idx, revert_padding_mask):
    # Mask token
    mask_token = torch.zeros([1])
    
    # Replace remain padding mask to mask token
    remain_data = torch.where(remain_padding_mask==1, remain_data, mask_token)
    
    # Make full data
    mask_token = mask_token.unsqueeze(0).repeat(remain_data.shape[0], masked_idx.shape[-1])
    full_data = torch.cat([remain_data, mask_token], dim=-1)
    
    # Revert
    reverted_data = torch.gather(full_data, index=revert_idx, dim=-1)
    reverted_data = torch.where(revert_padding_mask==1, reverted_data, -torch.inf)
    return reverted_data

reverted_arr = apply_revert(remain_arr, remain_padding_mask, masked_idx, revert_idx, revert_padding_mask)
print("_"*100)
print("reverted_arr:\n", reverted_arr)

arr:
 tensor([[ 2,  4,  0,  0,  0],
        [ 2,  4,  6,  0,  0],
        [ 2,  4,  6,  8,  0],
        [ 2,  4,  6,  8, 10]])
____________________________________________________________________________________________________
remain_arr:
 tensor([[ 4, 99],
        [ 4, 99],
        [ 4,  8],
        [ 2, 10]])
____________________________________________________________________________________________________
reverted_arr:
 tensor([[ 0.,  4., -inf, -inf, -inf],
        [ 0.,  4.,  0., -inf, -inf],
        [ 0.,  4.,  0.,  8., -inf],
        [ 2.,  0.,  0.,  0., 10.]])
